<a href="https://colab.research.google.com/github/hyunholee26/Short-term-water-level-prediction-in-the-river-using-LSTM-with-MC-dropout/blob/main/collect_data/%5Bwlp%5D_2_get_wamis_openapi_rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from urllib.request import Request, urlopen, build_opener, HTTPRedirectHandler, addinfourl
from urllib.parse import urlencode, quote_plus

from bs4 import BeautifulSoup

In [ ]:
path = '/content/drive/MyDrive/data/flood/'

In [ ]:
url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/rf_dubrfobs'
queryParams = '?' + urlencode({ quote_plus('key') : '36c9f67744f056ef568d114e3f9187aadda8173c82', quote_plus('oper') : 'y' })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8')
response_body
#soup = BeautifulSoup(response_body, "html.parser")

'{"result":{"code":"success","msg":"완료되었습니다!"},"count":769,"list":[{"bbsnnm":"한강","obscd":"10011100","obsnm":"대관령","clsyn":"운영","obsknd":"보통","sbsncd":"100105","mngorg":"기상청"},{"bbsnnm":"한강","obscd":"10011217","obsnm":"정선군","clsyn":"운영","obsknd":"보통","sbsncd":"100112","mngorg":"기상청"},{"bbsnnm":"한강","obscd":"10014010","obsnm":"평창군(월정분교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100108","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"10014020","obsnm":"정선군(정선군2청사)","clsyn":"운영","obsknd":"T/M","sbsncd":"100112","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"10014080","obsnm":"강릉시(대기리)","clsyn":"운영","obsknd":"T/M","sbsncd":"100106","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"10014090","obsnm":"정선군(고한읍사무소)","clsyn":"운영","obsknd":"T/M","sbsncd":"100113","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"10014100","obsnm":"삼척시(번천리)","clsyn":"운영","obsknd":"T/M","sbsncd":"100101","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"10014110","obsnm":"평창군(상진부리)","clsyn":"운영","obsknd":"T/M","sbsncd":"100108","mngorg":"한국수자원공사"},{"bbsnnm":"한강

In [ ]:
import json

jj = json.loads(response_body)
jj['list']

[{'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '기상청',
  'obscd': '10011100',
  'obsknd': '보통',
  'obsnm': '대관령',
  'sbsncd': '100105'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '기상청',
  'obscd': '10011217',
  'obsknd': '보통',
  'obsnm': '정선군',
  'sbsncd': '100112'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '10014010',
  'obsknd': 'T/M',
  'obsnm': '평창군(월정분교)',
  'sbsncd': '100108'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '10014020',
  'obsknd': 'T/M',
  'obsnm': '정선군(정선군2청사)',
  'sbsncd': '100112'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '10014080',
  'obsknd': 'T/M',
  'obsnm': '강릉시(대기리)',
  'sbsncd': '100106'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '10014090',
  'obsknd': 'T/M',
  'obsnm': '정선군(고한읍사무소)',
  'sbsncd': '100113'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '10014100',
  'obsknd': 'T/M',
  'obsnm': '삼척시(번천리)',
  'sbsncd': '100101'},
 {'bbsnnm':

In [ ]:
df = pd.json_normalize(jj['list'])
df

,bbsnnm,obscd,obsnm,clsyn,obsknd,sbsncd,mngorg
0,한강,10011100,대관령,운영,보통,100105,기상청
1,한강,10011217,정선군,운영,보통,100112,기상청
2,한강,10014010,평창군(월정분교),운영,T/M,100108,환경부
3,한강,10014020,정선군(정선군2청사),운영,T/M,100112,환경부
4,한강,10014080,강릉시(대기리),운영,T/M,100106,환경부
...,...,...,...,...,...,...,...
764,영산강서해,53031169,흑산도,운영,보통,530301,기상청
765,제주도,60011185,고산,운영,보통,600101,기상청
766,제주도,60021184,제주,운영,보통,600203,기상청
767,제주도,60031189,서귀포,운영,보통,600303,기상청


In [ ]:
class NoRedirectHandler(HTTPRedirectHandler):
    def http_error_302(self, req, fp, code, msg, headers):
        infourl = addinfourl(fp, headers, req.get_full_url())
        infourl.status = code
        infourl.code = code
        print('exception')
        raise Exception('redirection')
        return infourl
    http_error_300 = http_error_302
    http_error_301 = http_error_302
    http_error_303 = http_error_302
    http_error_307 = http_error_302

url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/rf_obsinfo'
df_detail = pd.DataFrame()
for i, row in df.iterrows():
  obscd = row['obscd'] 
  queryParams = '?' + urlencode({ quote_plus('key') : '36c9f67744f056ef568d114e3f9187aadda8173c82', quote_plus('obscd') : str(obscd) })
  request = Request(url + queryParams)
  request.get_method = lambda: 'GET'
  opener = build_opener(NoRedirectHandler())

  trycount = 10 # retry counter 
  while trycount > 0 :
    try:
      response_body = opener.open(request).read().decode('utf-8')
      break;
    except:
      trycount = trycount - 1
      continue;
  #response_body = urlopen(request).read().decode('utf-8')

  if trycount > 0:
    print(response_body)
    jj = json.loads(response_body)
    if i == 0:
      df_detail = pd.json_normalize(jj['list'])
    else:
      df_detail = df_detail.append(pd.json_normalize(jj['list']))
    #display(df_detail)

exception
{"result":{"code":"success","msg":"완료되었습니다!"},"count":1,"list":[{"obsnm":"대관령","obsnmeng":"100","obscd":"10011100","mngorg":"기상청","bbsnnm":"한강","sbsncd":"100105","opendt":"1971-07-15","obsknd":"지상","addr":"(산간)강원도 평창군 대관령면 횡계리 ","lon":"128.718","lat":"37.677","shgt":"772.57","hrdtstart":"자료없음","hrdtend":"자료없음","dydtstart":"자료없음","dydtend":"자료없음"}]}
{"result":{"code":"success","msg":"완료되었습니다!"},"count":1,"list":[{"obsnm":"정선군","obsnmeng":"217","obscd":"10011217","mngorg":"기상청","bbsnnm":"한강","sbsncd":"100112","opendt":"2010-08-06","obsknd":"지상","addr":"강원도 정선군 정선읍 북실리 ","lon":"128.646","lat":"37.381","shgt":"307.4","hrdtstart":"자료없음","hrdtend":"자료없음","dydtstart":"자료없음","dydtend":"자료없음"}]}
{"result":{"code":"success","msg":"완료되었습니다!"},"count":1,"list":[{"obsnm":"평창군(월정분교)","obsnmeng":"Pyeongchanggun(Woljeongbungyo)","obscd":"10014010","mngorg":"환경부","bbsnnm":"한강","sbsncd":"100108","opendt":"1916-01-01","obsknd":"T/M","addr":"강원도 평창군 진부면 내동산길 4 진부초등학교 월정분교(폐교)","lon":"128-35-57",

In [ ]:
df_detail

,obsnm,obsnmeng,obscd,mngorg,bbsnnm,sbsncd,opendt,obsknd,addr,lon,lat,shgt,hrdtstart,hrdtend,dydtstart,dydtend
0,대관령,100,10011100,기상청,한강,100105,1971-07-15,지상,(산간)강원도 평창군 대관령면 횡계리,128.718,37.677,772.57,자료없음,자료없음,자료없음,자료없음
0,정선군,217,10011217,기상청,한강,100112,2010-08-06,지상,강원도 정선군 정선읍 북실리,128.646,37.381,307.4,자료없음,자료없음,자료없음,자료없음
0,평창군(월정분교),Pyeongchanggun(Woljeongbungyo),10014010,환경부,한강,100108,1916-01-01,T/M,강원도 평창군 진부면 내동산길 4 진부초등학교 월정분교(폐교),128-35-57,37-42-34,614.2,19830701,20210817,19830101,20210817
0,정선군(정선군2청사),Jeongseongun(Jeongseongun2chungsa),10014020,환경부,한강,100112,1916-07-01,T/M,강원도 정선군 정선읍 녹송4길 71 정선군 제2청사,128-40-25,37-23-03,316.7,19800604,20210817,19650401,20210817
0,강릉시(대기리),Gangneungsi(Daegiri),10014080,환경부,한강,100106,1983-07-01,T/M,강원도 강릉시 왕산면 대기리 922-1,128-46-39,37-35-10,632.2,19830714,20210817,19830701,20210817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,흑산도,169,53031169,기상청,영산강서해,530301,1997-01-01,지상,전라남도 신안군 흑산면 예리,125.451,34.687,76.49,자료없음,자료없음,자료없음,자료없음
0,고산,185,60011185,기상청,제주도,600101,1988-01-01,지상,제주특별자치도 제주시 한경면 고산리,126.163,33.294,74.29,자료없음,자료없음,자료없음,자료없음
0,제주,184,60021184,기상청,제주도,600203,1923-05-01,지상,제주특별자치도 제주시 건입동,126.53,33.514,20.45,자료없음,자료없음,자료없음,자료없음
0,서귀포,189,60031189,기상청,제주도,600303,1961-01-01,지상,제주특별자치도 서귀포시 서귀동,126.565,33.246,48.96,자료없음,자료없음,자료없음,자료없음


In [ ]:
df_detail.shape

(768, 16)

In [ ]:
df_final = pd.merge(df, df_detail, left_on='obscd', right_on='obscd', how='left')
df_final

,bbsnnm_x,obscd,obsnm_x,clsyn,obsknd_x,sbsncd_x,mngorg_x,obsnm_y,obsnmeng,mngorg_y,bbsnnm_y,sbsncd_y,opendt,obsknd_y,addr,lon,lat,shgt,hrdtstart,hrdtend,dydtstart,dydtend
0,한강,10011100,대관령,운영,보통,100105,기상청,대관령,100,기상청,한강,100105,1971-07-15,지상,(산간)강원도 평창군 대관령면 횡계리,128.718,37.677,772.57,자료없음,자료없음,자료없음,자료없음
1,한강,10011217,정선군,운영,보통,100112,기상청,정선군,217,기상청,한강,100112,2010-08-06,지상,강원도 정선군 정선읍 북실리,128.646,37.381,307.4,자료없음,자료없음,자료없음,자료없음
2,한강,10014010,평창군(월정분교),운영,T/M,100108,환경부,평창군(월정분교),Pyeongchanggun(Woljeongbungyo),환경부,한강,100108,1916-01-01,T/M,강원도 평창군 진부면 내동산길 4 진부초등학교 월정분교(폐교),128-35-57,37-42-34,614.2,19830701,20210817,19830101,20210817
3,한강,10014020,정선군(정선군2청사),운영,T/M,100112,환경부,정선군(정선군2청사),Jeongseongun(Jeongseongun2chungsa),환경부,한강,100112,1916-07-01,T/M,강원도 정선군 정선읍 녹송4길 71 정선군 제2청사,128-40-25,37-23-03,316.7,19800604,20210817,19650401,20210817
4,한강,10014080,강릉시(대기리),운영,T/M,100106,환경부,강릉시(대기리),Gangneungsi(Daegiri),환경부,한강,100106,1983-07-01,T/M,강원도 강릉시 왕산면 대기리 922-1,128-46-39,37-35-10,632.2,19830714,20210817,19830701,20210817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,영산강서해,53031169,흑산도,운영,보통,530301,기상청,흑산도,169,기상청,영산강서해,530301,1997-01-01,지상,전라남도 신안군 흑산면 예리,125.451,34.687,76.49,자료없음,자료없음,자료없음,자료없음
765,제주도,60011185,고산,운영,보통,600101,기상청,고산,185,기상청,제주도,600101,1988-01-01,지상,제주특별자치도 제주시 한경면 고산리,126.163,33.294,74.29,자료없음,자료없음,자료없음,자료없음
766,제주도,60021184,제주,운영,보통,600203,기상청,제주,184,기상청,제주도,600203,1923-05-01,지상,제주특별자치도 제주시 건입동,126.53,33.514,20.45,자료없음,자료없음,자료없음,자료없음
767,제주도,60031189,서귀포,운영,보통,600303,기상청,서귀포,189,기상청,제주도,600303,1961-01-01,지상,제주특별자치도 서귀포시 서귀동,126.565,33.246,48.96,자료없음,자료없음,자료없음,자료없음


In [ ]:
df_final['lon_new'] = pd.to_numeric(df_final.lon.str.split('-').str[0], errors='coerce') + pd.to_numeric(df_final.lon.str.split('-').str[1], errors='coerce') / 60 + pd.to_numeric(df_final.lon.str.split('-').str[2], errors='coerce') / 3600
df_final['lat_new'] = pd.to_numeric(df_final.lat.str.split('-').str[0], errors='coerce') + pd.to_numeric(df_final.lat.str.split('-').str[1], errors='coerce') / 60 + pd.to_numeric(df_final.lat.str.split('-').str[2], errors='coerce') / 3600
df_final

,bbsnnm_x,obscd,obsnm_x,clsyn,obsknd_x,sbsncd_x,mngorg_x,obsnm_y,obsnmeng,mngorg_y,bbsnnm_y,sbsncd_y,opendt,obsknd_y,addr,lon,lat,shgt,hrdtstart,hrdtend,dydtstart,dydtend,lon_new,lat_new
0,한강,10011100,대관령,운영,보통,100105,기상청,대관령,100,기상청,한강,100105,1971-07-15,지상,(산간)강원도 평창군 대관령면 횡계리,128.718,37.677,772.57,자료없음,자료없음,자료없음,자료없음,NaN,NaN
1,한강,10011217,정선군,운영,보통,100112,기상청,정선군,217,기상청,한강,100112,2010-08-06,지상,강원도 정선군 정선읍 북실리,128.646,37.381,307.4,자료없음,자료없음,자료없음,자료없음,NaN,NaN
2,한강,10014010,평창군(월정분교),운영,T/M,100108,환경부,평창군(월정분교),Pyeongchanggun(Woljeongbungyo),환경부,한강,100108,1916-01-01,T/M,강원도 평창군 진부면 내동산길 4 진부초등학교 월정분교(폐교),128-35-57,37-42-34,614.2,19830701,20210817,19830101,20210817,128.599167,37.709444
3,한강,10014020,정선군(정선군2청사),운영,T/M,100112,환경부,정선군(정선군2청사),Jeongseongun(Jeongseongun2chungsa),환경부,한강,100112,1916-07-01,T/M,강원도 정선군 정선읍 녹송4길 71 정선군 제2청사,128-40-25,37-23-03,316.7,19800604,20210817,19650401,20210817,128.673611,37.384167
4,한강,10014080,강릉시(대기리),운영,T/M,100106,환경부,강릉시(대기리),Gangneungsi(Daegiri),환경부,한강,100106,1983-07-01,T/M,강원도 강릉시 왕산면 대기리 922-1,128-46-39,37-35-10,632.2,19830714,20210817,19830701,20210817,128.777500,37.586111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,영산강서해,53031169,흑산도,운영,보통,530301,기상청,흑산도,169,기상청,영산강서해,530301,1997-01-01,지상,전라남도 신안군 흑산면 예리,125.451,34.687,76.49,자료없음,자료없음,자료없음,자료없음,NaN,NaN
765,제주도,60011185,고산,운영,보통,600101,기상청,고산,185,기상청,제주도,600101,1988-01-01,지상,제주특별자치도 제주시 한경면 고산리,126.163,33.294,74.29,자료없음,자료없음,자료없음,자료없음,NaN,NaN
766,제주도,60021184,제주,운영,보통,600203,기상청,제주,184,기상청,제주도,600203,1923-05-01,지상,제주특별자치도 제주시 건입동,126.53,33.514,20.45,자료없음,자료없음,자료없음,자료없음,NaN,NaN
767,제주도,60031189,서귀포,운영,보통,600303,기상청,서귀포,189,기상청,제주도,600303,1961-01-01,지상,제주특별자치도 서귀포시 서귀동,126.565,33.246,48.96,자료없음,자료없음,자료없음,자료없음,NaN,NaN


In [ ]:
df_final.to_csv(path + 'rainfall_position.csv', index=False)

In [ ]:
import folium

#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
lat = df_final['lat_new'].mean()
lon = df_final['lon_new'].mean()

m = folium.Map([lat,lon],zoom_start=9)

for i in df_final.index:
    sub_lat = df_final.loc[i,'lat_new']
    sub_lon = df_final.loc[i,'lon_new']

    if (np.isnan(sub_lat) == False) & (np.isnan(sub_lat) == False) :
      title = df_final.loc[i,'obsnm_x'] + '(' + df_final.loc[i,'obscd'] + ')'
      folium.Marker([sub_lat,sub_lon],tooltip = title).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting 
m.save(path + 'example_rainfall.html')
m

